<a href="https://colab.research.google.com/github/mickeyrahm/Portfolio/blob/master/notebooks/starter_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# --- Load and prepare data ---
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes['dteday'] = pd.to_datetime(bikes['dteday'], errors='coerce')
bikes = bikes.dropna(subset=['dteday'])
bikes['total_riders'] = bikes['casual'] + bikes['registered']

# Feature engineering
bikes['hr_sin'] = np.sin(2 * np.pi * bikes['hr'] / 24)
bikes['hr_cos'] = np.cos(2 * np.pi * bikes['hr'] / 24)
bikes['day_of_week'] = bikes['dteday'].dt.dayofweek
bikes['dow_sin'] = np.sin(2 * np.pi * bikes['day_of_week'] / 7)
bikes['dow_cos'] = np.cos(2 * np.pi * bikes['day_of_week'] / 7)
bikes['month'] = bikes['dteday'].dt.month
bikes['month_sin'] = np.sin(2 * np.pi * bikes['month'] / 12)
bikes['month_cos'] = np.cos(2 * np.pi * bikes['month'] / 12)
bikes['temp_x_hum'] = bikes['temp_c'] * bikes['hum']
bikes['feels_x_wind'] = bikes['feels_like_c'] * bikes['windspeed']
bikes['working_hr'] = bikes['workingday'] * bikes['hr']
bikes['is_weekend'] = bikes['day_of_week'].isin([5, 6]).astype(int)
bikes['weekend_or_holiday'] = bikes['is_weekend'] | bikes['holiday']
bikes['time_of_day'] = pd.cut(bikes['hr'], bins=[-1, 6, 12, 18, 24],
                              labels=['Night', 'Morning', 'Afternoon', 'Evening'])
bikes = pd.get_dummies(bikes, columns=['season', 'weathersit', 'time_of_day'], drop_first=True)

# Define features and target
X = bikes.drop(columns=['casual', 'registered', 'total_riders', 'dteday'])
y = bikes['total_riders']

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Build final model ---
def build_model():
    model = keras.Sequential([
        layers.Dense(256),
        layers.LeakyReLU(negative_slope=0.01),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(128),
        layers.LeakyReLU(negative_slope=0.01),
        layers.Dropout(0.3),
        layers.Dense(64),
        layers.LeakyReLU(negative_slope=0.01),
        layers.Dense(1)
    ])
    return model

model = build_model()

# Compile using the optimal learning rate found
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Train the model
early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"\nFinal Test MAE: {test_mae:.2f}")


Epoch 1/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 64362.4297 - mae: 161.7187 - val_loss: 28460.6328 - val_mae: 106.1441
Epoch 2/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 33866.7852 - mae: 119.9211 - val_loss: 27115.6309 - val_mae: 104.1019
Epoch 3/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 31648.8633 - mae: 115.4135 - val_loss: 25717.9980 - val_mae: 102.0463
Epoch 4/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 29751.2383 - mae: 111.7112 - val_loss: 25095.6875 - val_mae: 99.8930
Epoch 5/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 29355.7832 - mae: 111.1102 - val_loss: 25237.8457 - val_mae: 100.5505
Epoch 6/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - loss: 29416.8672 - mae: 111.5488 - val_loss: 24777.3789 - val_mae: 99.2439
Epoch 7/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 29034.3047 - mae: 110.1228 - val_loss: 24861.6875 - val_mae: 99.5141
Epoch 8/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 2

In [45]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# --- Load and prepare data ---
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes['dteday'] = pd.to_datetime(bikes['dteday'], errors='coerce')
bikes = bikes.dropna(subset=['dteday'])
bikes['total_riders'] = bikes['casual'] + bikes['registered']

# Feature engineering
bikes['hr_sin'] = np.sin(2 * np.pi * bikes['hr'] / 24)
bikes['hr_cos'] = np.cos(2 * np.pi * bikes['hr'] / 24)
bikes['day_of_week'] = bikes['dteday'].dt.dayofweek
bikes['dow_sin'] = np.sin(2 * np.pi * bikes['day_of_week'] / 7)
bikes['dow_cos'] = np.cos(2 * np.pi * bikes['day_of_week'] / 7)
bikes['month'] = bikes['dteday'].dt.month
bikes['month_sin'] = np.sin(2 * np.pi * bikes['month'] / 12)
bikes['month_cos'] = np.cos(2 * np.pi * bikes['month'] / 12)
bikes['temp_x_hum'] = bikes['temp_c'] * bikes['hum']
bikes['feels_x_wind'] = bikes['feels_like_c'] * bikes['windspeed']
bikes['working_hr'] = bikes['workingday'] * bikes['hr']
bikes['is_weekend'] = bikes['day_of_week'].isin([5, 6]).astype(int)
bikes['weekend_or_holiday'] = bikes['is_weekend'] | bikes['holiday']
bikes['time_of_day'] = pd.cut(bikes['hr'], bins=[-1, 6, 12, 18, 24],
                              labels=['Night', 'Morning', 'Afternoon', 'Evening'])
bikes = pd.get_dummies(bikes, columns=['season', 'weathersit', 'time_of_day'], drop_first=True)

# Define features and target
X = bikes.drop(columns=['casual', 'registered', 'total_riders', 'dteday'])
y = bikes['total_riders']

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Optimizer comparison ---
optimizers = {
    "Adam": keras.optimizers.Adam(learning_rate=0.001),
    "RMSprop": keras.optimizers.RMSprop(learning_rate=0.001),
    "SGD": keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    "Adagrad": keras.optimizers.Adagrad(learning_rate=0.001),
    "Adadelta": keras.optimizers.Adadelta(learning_rate=1.0),
    "Nadam": keras.optimizers.Nadam(learning_rate=0.001)
}

results = []

for name, opt in optimizers.items():
    print(f"\n🔁 Training with optimizer: {name}")

    model = keras.Sequential([
        layers.Dense(256),
        layers.LeakyReLU(negative_slope=0.01),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(128),
        layers.LeakyReLU(negative_slope=0.01),
        layers.Dropout(0.3),
        layers.Dense(64),
        layers.LeakyReLU(negative_slope=0.01),
        layers.Dense(1)
    ])

    model.compile(optimizer=opt, loss='mse', metrics=['mae'])

    early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

    model.fit(
        X_train_scaled, y_train,
        validation_split=0.2,
        epochs=20,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )

    _, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)
    print(f"📉 Test MAE for {name}: {test_mae:.2f}")
    results.append((name, test_mae))

# --- Summary ---
print("\n🔚 Optimizer Results:")
for name, mae in results:
    print(f"{name:<10} → Test MAE: {mae:.2f}")



🔁 Training with optimizer: Adam
📉 Test MAE for Adam: 96.41

🔁 Training with optimizer: RMSprop
📉 Test MAE for RMSprop: 96.10

🔁 Training with optimizer: SGD
📉 Test MAE for SGD: nan

🔁 Training with optimizer: Adagrad
📉 Test MAE for Adagrad: 118.41

🔁 Training with optimizer: Adadelta
📉 Test MAE for Adadelta: 95.62

🔁 Training with optimizer: Nadam
📉 Test MAE for Nadam: 96.25

🔚 Optimizer Results:
Adam       → Test MAE: 96.41
RMSprop    → Test MAE: 96.10
SGD        → Test MAE: nan
Adagrad    → Test MAE: 118.41
Adadelta   → Test MAE: 95.62
Nadam      → Test MAE: 96.25
